## Project: Implementing a ChatGPT App with LangChain from Scratch



In [1]:
!pip install python-dotenv


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()  # By default, it looks for a `.env` file in the current directory


True

In [3]:
%pip install --upgrade -q langchain 
%pip install --upgrade -q langchain-community
%pip install --upgrade -q openai 



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name='gpt-4-turbo', temperature=1)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
#       SystemMessage(content="You are a chatbot having a conversation with a human."),
        SystemMessage(content='You respond only in Spanish.'),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

# chain = LLMChain(
#     llm=llm,
#     prompt=prompt,
#     verbose=True
# )

# if the code above gives an error use this:
chain = prompt | llm | StrOutputParser()

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit','q', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  Hello, How are you?


Hola, estoy bien, ¿y tú cómo estás?
--------------------------------------------------


Your prompt:  quit


Goodbye!


## Adding Chat Memory Using ConversationBufferMemory

In [7]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False)

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableMap

# Create the LLM
llm = ChatOpenAI(model_name='gpt-4-turbo', temperature=1)

# Create memory
memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

# Prompt template
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a chatbot having a conversation with a human."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{content}")
])

# Create chain using Runnable
chain = RunnableMap({
    "chat_history": memory.load_memory_variables,
    "content": lambda x: x["content"]
}) | prompt | llm

# Run loop
while True:
    content = input("Your prompt: ")
    if content.lower() in ["quit", "exit", "bye"]:
        print("Goodbye!")
        break

    # Invoke chain
    result = chain.invoke({"content": content})
    print(result.content)
    print('-' * 50)

    # Save to memory
    memory.save_context({"content": content}, {"output": result.content})


Your prompt:  quit


Goodbye!


## Saving Chat Sessions

In [8]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=False) 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain

# 1. Import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model_name='gpt-4-turbo', temperature=1)

# 2. Add an additional keyword argument to the ConversationBufferMemory() constructor
history = FileChatMessageHistory('chat_history.json')
memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})
    print(response)
    print('-' * 50)
    

Your prompt:  conversational buffer memory is


{'content': 'conversational buffer memory is', 'chat_history': [], 'text': '"Conversational buffer memory" in the context of chatbots and AI refers to the ability of an AI system to retain information from earlier in the conversation and use that information to make responses more relevant and coherent. This memory helps the AI understand the context of the conversation and maintain a flow, making interactions seem more natural to human users.\n\nDifferent implementations of AI and chatbots may utilize various approaches to manage conversational buffer memory. For instance, some might keep track of the conversation as long as the session is active (short-term memory), while others might have capabilities to remember past interactions over longer periods (long-term memory), though they typically do this using anonymized or generalized data to preserve user privacy.\n\nIn practice, this memory allows the chatbot to recall what has been discussed earlier in a conversation, preventing the 

Your prompt:  quit


Goodbye!


In [9]:
# The messages property contains the list of messages in order.
print(history.messages)

[HumanMessage(content='conversational buffer memory is', additional_kwargs={}, response_metadata={}), AIMessage(content='"Conversational buffer memory" in the context of chatbots and AI refers to the ability of an AI system to retain information from earlier in the conversation and use that information to make responses more relevant and coherent. This memory helps the AI understand the context of the conversation and maintain a flow, making interactions seem more natural to human users.\n\nDifferent implementations of AI and chatbots may utilize various approaches to manage conversational buffer memory. For instance, some might keep track of the conversation as long as the session is active (short-term memory), while others might have capabilities to remember past interactions over longer periods (long-term memory), though they typically do this using anonymized or generalized data to preserve user privacy.\n\nIn practice, this memory allows the chatbot to recall what has been discuss